In [1]:
import sys
import platform
from os.path import join, exists, abspath, dirname
from os import getcwd, makedirs
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colormaps, cm, colors
import scipy
from sklearn.metrics import r2_score
import seaborn as sns

from tqdm import tqdm

In [2]:
import nibabel as nb
from nilearn import plotting, image
from nipype.interfaces import fsl

In [3]:
dir_current = getcwd().replace('\\','/')

tmp = dir_current.split('/')
idx = [ii for ii, s in enumerate(tmp) if s=='github'][0]

dir_git = '/'.join(tmp[:idx+1])
dir_git

'/Users/sungbeenpark/github'

In [4]:
dname = join(dir_git,'nitools')
sys.path.append(dname)
import nitools as nt

In [5]:
dname = join(dir_git,'SUITPy')
sys.path.append(dname)
import SUITPy as suit

In [6]:
dname = join(dir_git)
sys.path.append(dname)
import surfAnalysisPy as surf

In [7]:
dname = join(dir_git,'Functional_Fusion')
sys.path.append(dname)
import Functional_Fusion.atlas_map as am
import Functional_Fusion.reliability as rel

In [8]:
dname = join(dir_git,'SeqSpatialSupp_fMRI')
sys.path.append(dname)
from SSS import deal_spm
from SSS import util as su
from SSS import plot as splt
from SSS import image as simage
from SSS import glmsingle as ssingle

---

In [9]:
list_sn = su.get_list_sn()
list_sn

array(['01', '02', '03', '05', '06', '08', '09', '10', '11', '12', '13',
       '14'], dtype='<U2')

---

In [10]:
atlasDir = su.get_dir_atlas()

In [11]:
# hem = ['L', 'R']
h = 'L'

In [12]:
list_roi = ['S1', 'M1', 'PMd', 'PMv', 'SMA', 'V1', 'SPLa', 'SPLp']
list_label = [1, 2, 3, 4, 5, 6, 7, 8]

---

In [13]:
glm = 1

---

### GLM original

In [14]:
dir_glm = su.get_dir_glm(glm)
dir_glm

'/Volumes/Diedrichsen_data$/data/SeqSpatialSupp_fMRI/glm_1'

---

### GLMsingle

In [16]:
dir_glm = ssingle.get_dir_glmsingle(glm)
dir_glm

'/Volumes/Diedrichsen_data$/data/SeqSpatialSupp_fMRI/GLMsingle/glm_1'

---

In [ ]:
for sn in tqdm(list_sn):
    subj = 'S'+sn
    reginfo = deal_spm.load_reginfo(subj=subj, dir_glm=dir_glm)
    part_vec = reginfo.run.values.flatten()
    if 'GLMsingle' in dir_glm:
        cond_vec = reginfo.name.values.flatten()
    else:
        cond_vec = reginfo.reg.values.flatten()
    list_nii = sorted(glob(join(dir_glm,subj,'beta_*.nii')))
    white, pial, mask = simage.get_WPM(subj=subj, glm=glm)
    dataset = []
    for roi, label in zip(list_roi, list_label):
        atlas, ainfo = am.get_atlas('fs32k_%s'%h)
        subatlas = atlas.get_subatlas_image(
            mask_img=join(atlasDir,'ROI.32k.%s.label.gii'%h),
            value=label
        )
        amap = am.AtlasMapSurf(
            vertex=subatlas.vertex[0],
            white_surf=white,
            pial_surf=pial,
            mask_img=mask
        )
        
        ## Compute the voxels in native space
        amap.build()
        
        ## This extract all the relevant voxels in native space (use for RSA)
        beta = amap.extract_data_native(list_nii)
        idx_nan = np.isnan(beta).any(axis=0)
        beta = beta[:, ~idx_nan]
        dataset.append(beta)

    ## save the data of hem and region:
    ## Convert to object array to handle different shapes
    dataset_obj = np.empty(len(dataset), dtype=object)
    for ii, data in enumerate(dataset):
        dataset_obj[ii] = data

    np.savez(
        file=join(dir_glm,'beta.%s.%s.%drois.npz'%(h,subj,len(list_roi))), 
        dataset=dataset_obj,
        cond_vec=cond_vec,
        part_vec=part_vec,
        subj=subj
    )

  8%|████████▍                                                                                            | 1/12 [06:51<1:15:27, 411.60s/it]